In [2]:
from coredotfinance.data import KrxReader
import pandas as pd
import numpy as np
import requests
import lxml
from bs4 import BeautifulSoup
from requests import Response
import re
import matplotlib.pyplot as plt
from pykrx import stock
from pykrx import bond
import time
import math
import seaborn as sns
import plotly.express as px

In [ ]:

#네이버 파이낸스 크롤링 for 매출액 but 전체 시장으로만 판단할수 있고 KOSPI, KOSDAQ 구분 X
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

finance_url = "https://finance.naver.com/sise/sise_group.naver?type=upjong"
res = requests.get(finance_url, headers=headers)
page_soup = BeautifulSoup(res.text, "lxml")
summary_html = page_soup.select('td a')

upjong_list = pd.DataFrame(re.findall('no=(.+?)">(.+?)</a>',str(summary_html)))
upjong_list = upjong_list.rename(columns={0:"no",1:"upjong"}).sort_values(by=['upjong']).reset_index(drop=True)
upjong_list.no = upjong_list.no.str.zfill(3)

request_upjong_url = "https://finance.naver.com/sise/field_submit.naver?menu=upjong&returnUrl=http%3A%2F%2Ffinance.naver.com%2Fsise%2Fsise_group_detail.naver%3Ftype%3Dupjong%26no%3D"
request_url1 = "&fieldIds=market_sum&fieldIds=property_total&fieldIds=debt_total&fieldIds=sales"
request_url2 = "&fieldIds=operating_profit&fieldIds=net_income&fieldIds=eps&fieldIds=dividend"

upjong = pd.DataFrame()
for i in (upjong_list['no'].to_list()):
    temp = pd.DataFrame()
    for j in ([request_url1,request_url2]):
        res = requests.get(request_upjong_url+i+j, headers=headers)
        page_soup = BeautifulSoup(res.text, "lxml")
        upjong_data_html = page_soup.select_one('#contentarea > div:nth-child(5)')
        col = [item.get_text().strip() for item in upjong_data_html.select('thead th')]
        col = [x for x in col if x not in ['종목명','토론실']]
        row = [item.get_text().strip() for item in upjong_data_html.select('tbody a')]
        row = list(filter(None,row))
        num = np.array([item.get_text().strip() for item in upjong_data_html.select('td.number')])
        num.resize(len(row),len(col))
        df = pd.DataFrame(num, columns=col,index=row).drop(columns=['현재가','전일비','등락률'])
        df['Sector'] = upjong_list[upjong_list.no==i].upjong.iat[0]
        temp = pd.concat([temp,df], axis = 1)

    upjong = pd.concat([upjong,temp], axis = 0)
    
with pd.ExcelWriter('upjong.xlsx', engine='openpyxl') as writer:
    upjong.to_excel(writer)

In [3]:
krx = KrxReader()
company_df = krx.listed_company()
dataframe = krx.read_date('2022-07-08', kor=True)
listed_company_df = pd.merge(company_df, dataframe, on='종목코드', how='inner')

In [47]:
marketData = pd.read_excel('marketData.xlsx', engine='openpyxl')

def draw_bar(x, y, title=" "):
    fig = px.bar(x=x, y=y, title=title,height=350, width=1000)
    fig.show()  

def draw_hist(x_values, x_label=" "):
    fig = px.histogram(x_values, x=x_label, height=350, width=600)
    fig.show()
    
market = "KOSDAQ"
filtered_companies = marketData[['종목명', '시장구분', 'Sector', '시가총액']]
#KOSPI 시가총액 분포
companies = filtered_companies[filtered_companies['시장구분'] == market].sort_values(by=['시가총액'])
draw_hist(companies['시가총액'], "시가총액")


In [49]:
#KOSPI or KOSDAQ시가총액 합과 평균
companies = filtered_companies[filtered_companies['시장구분'] == market].sort_values(by=['시가총액'])
companies = companies.loc[companies['시가총액'] < 2.0*10**11] #상위 시가총액을 가진 기업 필터링 KOSPI 10**12, KOSDAQ 10**11

print(companies)
#draw_hist(companies, "시가총액")

groupby_sum = companies.groupby('Sector')['시가총액'].agg(시가총액='sum', MyCount='count').sort_values(by=['시가총액'])
groupby_mean = companies.groupby('Sector').mean().sort_values(by=['시가총액'])

groupby_sum = groupby_sum[-10:]
groupby_mean = groupby_mean[-10:]

draw_bar(groupby_sum.index.to_list(), groupby_sum['시가총액'].to_list(), title='20조이하 기업들의 업종별 시가총액 합')
draw_bar(groupby_mean.index.to_list(), groupby_mean['시가총액'].to_list(), title='20조이하 기업들의 업종별 시가총액 평균')

             종목명    시장구분     Sector          시가총액
2194     한국코퍼레이션  KOSDAQ  상업서비스와공급품    4207455498
79    IBKS제17호스팩  KOSDAQ       창업투자    6174400000
1867   케이프이에스제4호  KOSDAQ       창업투자    6364035000
2131   하나금융20호스팩  KOSDAQ       창업투자    6862500000
1563      유진스팩6호  KOSDAQ       창업투자    6885600000
...          ...     ...        ...           ...
1623       이원컴포텍  KOSDAQ      자동차부품  196705579710
147          NEW  KOSDAQ  방송과엔터테인먼트  197296169420
2111    피에스케이홀딩스  KOSDAQ  반도체와반도체장비  197511538200
1587         이녹스  KOSDAQ   레저용장비와제품  198510600600
1977          태웅  KOSDAQ  에너지장비및서비스  199873736190

[1154 rows x 4 columns]


In [67]:
#코스피 or 코스닥 매출액 합과 평균
filtered_companies = marketData[['종목명', 'Sector','시장구분','시가총액','영업이익','매출액']]
companies = filtered_companies[filtered_companies['시장구분'] == market]
print(companies.head())

companies = companies.dropna()
companies['영업이익'] = companies['영업이익'].str.replace(',','').astype(int)
companies['매출액'] = companies['매출액'].str.replace(',','').astype(int)

companies = companies.dropna()
companies = companies.sort_values(by=['매출액'])
#companies = companies.loc[companies['매출액'] < 10**6] 

groupby_sum = companies.groupby('Sector')['매출액'].agg(매출액='sum').sort_values(by=['매출액'])#,  MyCount='count'
groupby_mean = companies.groupby('Sector')['매출액'].agg(매출액='mean').sort_values(by=['매출액'])#,  MyCount='count'

groupby_sum = groupby_sum[-10:]
groupby_mean = groupby_mean[-10:]

draw_bar(groupby_sum.index.to_list(), groupby_sum['매출액'].to_list(), title= '%s 업종별 매출액 합'%market)
draw_bar(groupby_mean.index.to_list(), groupby_mean['매출액'].to_list(), title='%s 업종별 매출액 평균'%market)

      종목명      Sector    시장구분          시가총액 영업이익    매출액
0      3S          기계  KOSDAQ  136500963350   15    271
3  APS홀딩스  디스플레이장비및부품  KOSDAQ  167640496620  -40    394
4   AP시스템  디스플레이장비및부품  KOSDAQ  263604512250  643  5,287
5    AP위성         핸드셋  KOSDAQ  175708841600    8    402
8    BNGT    컴퓨터와주변기기  KOSDAQ  120352455875    1    722
